In [18]:
import pandas as pd
import numpy as np
from feature_engine.timeseries.forecasting import LagFeatures
# from feature_engine.

In [58]:
df = pd.read_csv('./data/pollution.csv',index_col=['date'], parse_dates=['date'])
df.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [59]:
df = df.pollution.to_frame().resample('D').mean()
df.head()

,pollution
date,
2010-01-02,145.958333
2010-01-03,78.833333
2010-01-04,31.333333
2010-01-05,42.458333
2010-01-06,56.416667


# lag

In [61]:
df1 = df.copy()
lag= 2 
df1[f'pollution_lag_{lag}'] = df1['pollution'].shift(periods=lag, freq='D')
df1.head()

,pollution,pollution_lag_2
date,,
2010-01-02,145.958333,NaN
2010-01-03,78.833333,NaN
2010-01-04,31.333333,145.958333
2010-01-05,42.458333,78.833333
2010-01-06,56.416667,31.333333


## feature engine

In [62]:
from feature_engine.timeseries.forecasting import LagFeatures
df2 = df.copy()
lag_transformer = LagFeatures(variables=['pollution'], freq=['1D','2D'])
lag_transformer.fit_transform(df2)

,pollution,pollution_lag_1D,pollution_lag_2D
date,,,
2010-01-02,145.958333,NaN,NaN
2010-01-03,78.833333,145.958333,NaN
2010-01-04,31.333333,78.833333,145.958333
2010-01-05,42.458333,31.333333,78.833333
2010-01-06,56.416667,42.458333,31.333333
...,...,...,...
2014-12-27,238.666667,170.250000,27.000000
2014-12-28,197.375000,238.666667,170.250000
2014-12-29,159.000000,197.375000,238.666667


# window 

In [63]:
df1 = df.copy()

In [64]:
window_size = 5
result = (df1['pollution'].rolling(window=window_size, min_periods=1)
                        .agg(['mean','std'])
                        .shift(freq='1D'))
result.head()

,mean,std
date,,
2010-01-03,145.958333,NaN
2010-01-04,112.395833,47.464543
2010-01-05,85.375000,57.591820
2010-01-06,74.645833,51.688218
2010-01-07,71.000000,45.499609


In [65]:
result = result.add_prefix(f'window_{window_size}_')
result.head()

,window_5_mean,window_5_std
date,,
2010-01-03,145.958333,NaN
2010-01-04,112.395833,47.464543
2010-01-05,85.375000,57.591820
2010-01-06,74.645833,51.688218
2010-01-07,71.000000,45.499609


In [66]:
df_window = df1.merge(result,how='left',right_index=True,left_index=True)
df_window.head()

,pollution,window_5_mean,window_5_std
date,,,
2010-01-02,145.958333,NaN,NaN
2010-01-03,78.833333,145.958333,NaN
2010-01-04,31.333333,112.395833,47.464543
2010-01-05,42.458333,85.375000,57.591820
2010-01-06,56.416667,74.645833,51.688218


## feature engine

In [69]:
df1 = df.copy()

In [73]:
from feature_engine.timeseries.forecasting import WindowFeatures
window_transformer = WindowFeatures(variables=['pollution'],
                                    functions=['mean','std'],
                                    window= [2,3,4],
                                   freq='D')

df_window = window_transformer.fit_transform(df1)
df_window.head(10)

,pollution,pollution_window_2_mean,pollution_window_2_std,pollution_window_3_mean,pollution_window_3_std,pollution_window_4_mean,pollution_window_4_std
date,,,,,,,
2010-01-02,145.958333,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,78.833333,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,31.333333,112.395833,47.464543,NaN,NaN,NaN,NaN
2010-01-05,42.458333,55.083333,33.587572,85.375000,57.591820,NaN,NaN
2010-01-06,56.416667,36.895833,7.866563,50.875000,24.843363,74.645833,51.688218
2010-01-07,69.000000,49.437500,9.870032,43.402778,12.568309,52.260417,20.472892
2010-01-08,176.208333,62.708333,8.897760,55.958333,13.276768,49.802083,16.404655
2010-01-09,88.500000,122.604167,75.807739,100.541667,65.830604,86.020833,61.094441
2010-01-10,57.250000,132.354167,62.019157,111.236111,57.106083,97.531250,54.086622


# scaler